In [1]:
from google.colab import drive; drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#pip install --upgrade tensorflow-gpu
#!pip install Keras==2.2.4
!pip install tokenizers

In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
#import sentencepiece as spm
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
#from tokenizers import ByteLevelBPETokenizer
from keras import backend as K
print("Tf",tf.__version__)

Tf 2.3.0


In [4]:
pip install transformers

In [5]:
from transformers import *

In [6]:
  from transformers import XLMRobertaTokenizer
  from transformers import XLMRobertaModel

In [7]:
MAX_LEN = 128  
AUTO = tf.data.experimental.AUTOTUNE
PATH = "/content/drive/My Drive/Gemastik/XLMRoberta"

tokenizer = XLMRobertaTokenizer.from_pretrained("jplu/tf-xlm-roberta-large",lowercase = True)

In [8]:

traindf = pd.read_excel("/content/drive/My Drive/hasil_preprocessing_kamusKita_kamusInternet (1).xlsx")
traindf.drop(["Unnamed: 0"],axis = 1,inplace = True)
 #traindf = pd.read_csv("/content/drive/My Drive/Gemastik/DatasetLabeled.csv",delimiter = "\t")
 #traindf.head()
 #traindf5000 = traindf[:5000]

In [9]:
traindf = traindf[["hasil_akhir_preprocessing","sentiment"]]
traindf.dropna(inplace = True)
traindf["sentiment"] = traindf["sentiment"].astype('float32')
print(len(traindf))
traindf.head()

3649


,hasil_akhir_preprocessing,sentiment
2,stayathome withmestudy bahas uji sekolah onlen,1.0
3,nasehat untuk pemuda ini rupa daily voice lett...,1.0
4,dari kampusny memang sudah ada elearning broth...,1.0
5,hmm kemarin teman kelas telepon iya inti kenap...,0.0
7,serius stres banget ngadepin ajar jarak jauh ini,0.0


In [10]:
def regular_encode(texts, tokenizer = tokenizer, maxlen=128):
    enc_di = tokenizer.batch_encode_plus(
        texts, 
        max_length=maxlen,
        return_attention_mask = False,
        return_token_type_ids = False,
        pad_to_max_length = True,
        truncation = True
        
    )
    
    return np.array(enc_di["input_ids"])

In [11]:
traindfonlyBalancePos = traindf[traindf["sentiment"] == 1]
traindfonlyBalanceNeg = traindf[traindf["sentiment"] == 0] 
traindfonlyBalanceNeg = traindfonlyBalanceNeg[:1040]
BalanceDf = pd.concat([traindfonlyBalancePos,traindfonlyBalanceNeg])

BalanceDf = BalanceDf.sample(frac=1).reset_index(drop=True)
(len(traindfonlyBalanceNeg) , len(traindfonlyBalancePos))
BalanceDf.head()

,hasil_akhir_preprocessing,sentiment
0,guru gerak kim web kukar gelar ajar daring lal...,1.0
1,ket mau tweet e takon sekolah sahaja yang tanya,1.0
2,selamat siang universitas buka adalah guru tin...,1.0
3,ini adik aku yang marah gara-gara sekolah engg...,0.0
4,yakali nanti kenangannyaa social,1.0


In [12]:
BalanceDf["Encoded"] = BalanceDf["hasil_akhir_preprocessing"].apply(regular_encode)
BalanceDf.loc[0,"Encoded"]

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1770: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


array([[   0,  706,    2, ...,    1,    1,    1],
       [   0,   75,    2, ...,    1,    1,    1],
       [   0, 1690,    2, ...,    1,    1,    1],
       ...,
       [   0,   81,    2, ...,    1,    1,    1],
       [   0,   17,    2, ...,    1,    1,    1],
       [   0,   10,    2, ...,    1,    1,    1]])

In [13]:
x_train,x_val,y_train,y_val = train_test_split(BalanceDf["hasil_akhir_preprocessing"],BalanceDf["sentiment"],
                                               test_size = 0.25, random_state = 0)
x_Encoded = regular_encode(x_train,tokenizer,MAX_LEN)
x_valEncoded = regular_encode(x_val,tokenizer,MAX_LEN)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1770: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [14]:
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train,num_classes = 2)
y_val = to_categorical(y_val,num_classes = 2)

In [15]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_Encoded, y_train))
    .batch(32)
    .repeat()
    .prefetch(AUTO)
)

In [16]:
valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_valEncoded, y_val))
    .batch(32)
    .cache()
    .prefetch(AUTO)
)

In [17]:
from transformers import TFAutoModel

In [23]:
import keras
def main_model():
  config = XLMRobertaConfig.from_pretrained("jplu/tf-xlm-roberta-large")
  config.num_labels = 2
  METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
]  
  strategy = tf.distribute.get_strategy()

  with strategy.scope():
    ENCODER = TFAutoModel.from_pretrained("jplu/tf-xlm-roberta-base")
    input_ids = tf.keras.layers.Input(shape=(MAX_LEN), dtype=tf.int32)
    input_masks = tf.keras.layers.Input(shape = (MAX_LEN), dtype= tf.int32 , name = "input_maks")
    input_Token = tf.keras.layers.Input(shape = (MAX_LEN), dtype= tf.int32 , name = "input_Token_type")

    inputs = [input_ids, input_masks, input_Token] 

      # Add a hidden layer
      #x = tf.keras.layers.Dense(units=768, activation='relu')(pooled_output)
      #x = tf.keras.layers.Dropout(0.1)(x)

      # Add output layer
      #outputs = Dense(1, activation="sigmoid")(x)# Construct a new model


    embedding, sequence_output = ENCODER(input_ids)
      #Disini Boleh Di adjust  
    logits = embedding[:,0,:]
    logits = tf.keras.layers.Dropout(0.2)(logits)
    logits = tf.keras.layers.Dense(64,"relu")(logits)
    logits = tf.keras.layers.Dropout(0.4)(logits)
    outputs = tf.keras.layers.Dense(2, activation="softmax", name="final_output")(logits)
    model = tf.keras.Model(inputs=input_ids, outputs=outputs)

    optimizer = tf.keras.optimizers.Adam(lr=1e-5)
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

      #for layer in model.layers[:2]:
      #  layer.trainable = False

    model.compile(optimizer=optimizer, loss=loss, metrics=METRICS)
  return model

model = main_model()
model.summary()

Some weights of the model checkpoint at jplu/tf-xlm-roberta-base were not used when initializing TFXLMRobertaModel: ['lm_head']
- This IS expected if you are initializing TFXLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFXLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFXLMRobertaModel were initialized from the model checkpoint at jplu/tf-xlm-roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLMRobertaModel for predictions without further training.


Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 128)]             0         
_________________________________________________________________
tfxlm_roberta_model_2 (TFXLM ((None, 128, 768), (None, 278043648 
_________________________________________________________________
tf_op_layer_strided_slice_2  [(None, 768)]             0         
_________________________________________________________________
dropout_118 (Dropout)        (None, 768)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                49216     
_________________________________________________________________
dropout_119 (Dropout)        (None, 64)                0         
_________________________________________________________________
final_output (Dense)         (None, 2)                

In [19]:
def create_learning_rate_scheduler(max_learn_rate=1e-5,
                                   end_learn_rate=1e-7,
                                   warmup_epoch_count=10,
                                   total_epoch_count=90):

    def lr_scheduler(epoch):
        if epoch < warmup_epoch_count:
            res = (max_learn_rate/warmup_epoch_count) * (epoch + 1)
        else:
            res = max_learn_rate*math.exp(math.log(end_learn_rate/max_learn_rate)*(epoch-warmup_epoch_count+1)/(total_epoch_count-warmup_epoch_count+1))
        return float(res)
    learning_rate_scheduler = tf.keras.callbacks.LearningRateScheduler(lr_scheduler, verbose=1)

    return learning_rate_scheduler

In [ ]:

total_epoch_count = 50
n_steps = x_train.shape[0]
history = model.fit(train_dataset,
          validation_data=valid_dataset,
          steps_per_epoch =n_steps,
          epochs=total_epoch_count)


Epoch 1/50
  70/1560 [>.............................] - ETA: 12:24 - loss: 0.7409 - tp: 1087.0000 - fp: 1145.0000 - tn: 1087.0000 - fn: 1145.0000 - accuracy: 0.4870 - precision: 0.4870 - recall: 0.4870 - auc: 0.4834

In [ ]:
x_train.shape

In [ ]:
#model_save_location = "/content/drive/My Drive/Gemastik/XLMROBERTA.h5"

In [ ]:
#model.save_weights(model_save_location, overwrite=True)

In [ ]:
label = np.array(traindf["sentiment"]).reshape(-1,1)
print(label)

In [ ]:
def compute_freqs(labels):
  '''
  Purpose : "compute Positive and negative frequencies"
  Args : labels(np.array): size(num_examples,num_class)
  
  returns positive_frequencies and Negative Frequencies
  '''
  labels = labels.reshape(-1,1)
  N = labels.shape[0]
  positive_frequencies = np.sum(labels,axis = 0) /  N
  negative_frequencies = 1 - positive_frequencies 

  return positive_frequencies,negative_frequencies


In [ ]:
test_pos,test_neg= compute_freqs(label)
print(f"Pos freqs : {test_pos}")
print(f"Neg Freqs: {test_neg}")

In [ ]:
freq_pos,freq_neg = compute_freqs(y_train.values)
print(f"Pos freqs : {freq_pos}")
print(f"Neg Freqs : {freq_neg}")

class_weight = {1: freq_neg, 0: freq_pos}

In [ ]:
neg = traindf["sentiment"][traindf["sentiment"] == 0].value_counts().values[0]
pos = traindf["sentiment"][traindf["sentiment"] == 1].value_counts().values[0]
total = pos + neg
weight_for_0 = (1 / neg) *(total)
weight_for_1 = (1/ pos) *(total)

print(total)

class_weight = {0: weight_for_0, 1: weight_for_1}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))

In [ ]:
from tensorflow.keras.models import load_model
#model.load_weights(filepath="/content/drive/My Drive/Gemastik/XLMROBERTA.h5")
Preloadedmodel = model
def FineTuningModel(models):
  METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
]
  #dditionalLayer = tf.keras.layers.Dense(2048,activation = 'tanh')
  models.layers[-2] = models.layers[-2].trainable = True
  models.layers[-4] = models.layers[-4].trainable = True
  #models.layers[-4] = additionalLayer
  loss = tf.keras.losses.BinaryCrossentropy()
  optimizer = tf.keras.optimizers.Adam(lr=2e-5)

  models.compile(optimizer = optimizer , loss = loss ,metrics = METRICS)

  return models

FineTunedModel = FineTuningModel(Preloadedmodel)
FineTunedModel.summary()


In [ ]:
x_train,x_val,y_train,y_val = train_test_split(traindf["hasil_akhir_preprocessing"],traindf["sentiment"],
                                               test_size = 0.25, random_state = 0)
x_train = regular_encode(x_train,tokenizer,MAX_LEN)
x_val = regular_encode(x_val,tokenizer,MAX_LEN)
print(x_train.shape,x_val.shape,y_train.shape)

In [ ]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_train, y_train))
    .repeat()
    .shuffle(2048)
    .batch(64)
    .prefetch(AUTO)
)

valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_val, y_val))
    .batch(64)
    .cache()
    .prefetch(AUTO)
)

In [ ]:
def class_weighted_pixelwise_crossentropy(target, output):
    output = tf.clip_by_value(output, 10e-8, 1.-10e-8)
    with open('class_weights.pickle', 'rb') as f:
        weight = pickle.load(f)
    return -tf.reduce_sum(target * weight * tf.log(output))

In [ ]:
BATCH_SIZE = 1024 
ITERATION = 3649 / BATCH_SIZE # tOTAL DATASET / bATCHSIZE
total_epoch_count = round(BATCH_SIZE) * round(ITERATION)
n_steps = x_train.shape[0]
FineTunedModel.fit(x = x_train,y = y_train,
          shuffle = True,
          validation_data=valid_dataset,
          epochs=total_epoch_count, 
          workers = -1,
          batch_size = 32,
          callbacks=[create_learning_rate_scheduler(max_learn_rate=1e-5,
                                                    end_learn_rate=1e-7,
                                                    warmup_epoch_count=20,
                                                    total_epoch_count=total_epoch_count),
                     keras.callbacks.EarlyStopping(patience=5),
                     ]) 

In [ ]:
#pip install numba
#from numba import cuda 
#device = cuda.get_current_device()
#device.reset()

In [ ]:
#model.evaluate(x_val,y_val)